In [2]:
import cv2
import numpy as np
import time
import Vehicles
import pandas as pd 

In [3]:
vehicle_up = 0
vehicle_down = 0
vehicles = [] # list of objects 

max_vehicle_weight = 5 
vehicle_id = 1 


df1_column_names = ["Id", "Direction", "Timestamp"]
df1 = pd.DataFrame(columns = df1_column_names)

df2_column_names = ["Flow", "Timestamp"]
df1 = pd.DataFrame(columns = df2_column_names)

In [4]:
cap = cv2.VideoCapture('video1.avi')
w = cap.get(3)
h = cap.get(4)
print(w)
frameArea = h*w
detection_area= frameArea/320
detector_up = np.int(2*(h/5)) 
detector_down = np.int(3*(h/5))
# define tracking area 
up_limit = np.int(1*(h/5)) # fifth of the horizon would be enough 
down_limit = np.int(4*(h/5))
# determine the area between the four lines 

area1 = np.array([[0,detector_down],[w,detector_down]], np.int32).reshape((-1,1,2))
area2 = np.array([[0,detector_up],[w,detector_up]], np.int32).reshape((-1,1,2))
area3 = np.array([[0,up_limit],[w,up_limit]], np.int32).reshape((-1,1,2))
area4 = np.array([[0,down_limit],[w,down_limit]], np.int32).reshape((-1,1,2))
background_substractor = cv2.createBackgroundSubtractorMOG2()
while(cap.isOpened()):
    _, frame = cap.read()
    for i in vehicles:
        i.get_weight()  
    background_substractor_mask = background_substractor.apply(frame)
    try:
        cv2.imshow('First Frame', frame)
        cv2.imshow('Backgroud Subtraction', background_substractor_mask)
        # apply threshold on the mask
        _, imBin = cv2.threshold(background_substractor_mask, 200, 255, cv2.THRESH_BINARY)
        # remove noise 
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN,np.ones((3,3), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((11,11), np.uint8)) 
        cv2.imshow('Masked Image', cv2.resize(mask, (400, 400)))
    except:
        #If there is no more frames to show...
        print('EOF')
        print ('UP:', vehicle_up)
        print ('DOWN:', vehicle_down)
        break
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
         contour_area = cv2.contourArea(cnt)
         # determine the distance to start tracking 
         if contour_area > detection_area and contour_area < 20000:
             # determine the center of the contours
            M = cv2.moments(cnt)
             # get the cordinates of the center
            center_x = int(M['m10']/M['m00'])
            center_y = int(M['m01']/M['m00'])
            # calculate the up-right bounding rectangle
            x,y,w,h = cv2.boundingRect(cnt)
            # try to seperate two close objects 
            new = True
            for i in vehicles:
                if abs(x-i.getX()) <= w and abs(y-i.getY()) <= h:
                    # we are still inside the dimentions of the same contour
                    new = False
                    # Update the coordinates of the vehicle and reset weight 
                    i.updateCoords(center_x,center_y)
                    if i.going_UP(detector_down, detector_up) == True:
                        vehicle_up += 1
                        #df1.append(i.getId(),"up",time.strftime("%c"))
                    elif i.going_DOWN(detector_down, detector_up) == True:
                        vehicle_down += 1  
                    #df1.append(i.getId(),"down",time.strftime("%c"))
                    break  
                if i.getState() == '1':
                    if i.getDir() == 'down' and i.getY() > down_limit:
                        i.setDone()
                    elif i.getDir() == 'up' and i.getY() < up_limit:
                        i.setDone()
                if i.timedOut():
                    # Remove from the list
                    index = vehicles.index(i)
                    vehicles.pop(index)
                    del i
            if new == True:
                p = Vehicles.Vehicles(vehicle_id,center_x,center_y, max_vehicle_weight)
                vehicles.append(p)
                vehicle_id += 1   
            # draw rectangle    
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2) 
    frame = cv2.polylines(frame, [area1], False, (255,0,0), thickness=2)
    frame = cv2.polylines(frame, [area2], False, (0,0,255), thickness=2)
    frame = cv2.polylines(frame, [area3], False, (255,255,255), thickness=1)
    frame = cv2.polylines(frame, [area4], False, (255,255,255), thickness=1)
    cv2.imshow('Frame', cv2.resize(frame, (400, 400)))    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break   
cap.release()
cv2.destroyAllWindows()  



                                                    


320.0


/tmp/ipykernel_2735/1698342973.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  detector_up = np.int(2*(h/5))
/tmp/ipykernel_2735/1698342973.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for mo

EOF
UP: 30
DOWN: 8


In [5]:
model.summary

NameError: name 'model' is not defined